In [1]:
import pandas as pd

In [2]:
df_test = pd.read_csv("data/hackathon_loreal_test_set.csv")
df_test

,Unnamed: 0,index,text
0,4538,4538,Want the low down on the Estee Edit I have a b...
1,764,764,President Trump may be the most high-profile h...
2,7008,7008,Sonic Silver Metallic Used 2017 Mazda CX-5 ava...
3,6395,6395,Discover the SPF formula that fits seamlessly ...
4,9282,9282,"LOS ANGELES Nov. 18, 2020 /PRNewswire/ -- Serv..."
...,...,...,...
2638,892,892,"""I love you people but I've been doing this si..."
2639,2581,2581,🍑Peachy Perfect🍑 . A rich chocolate sponge fil...
2640,10665,10665,⚡️PRESS PLAY⚡️ Metal Matte Liquid Lipsticks 💄⚡...
2641,3775,3775,Seen this first impression ➡️ 🔥 leave your TOP...
